In [1]:
!pip install pymupdf
!pip install sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [2]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import fitz  # PyMuPDF

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

def preprocess_text_modified(text):

    # To convert bullet points into full stops if they follow a semi-colon or start of the text
    text = re.sub(r'(?<=;)\s*●\s*', '. ', text)
    text = re.sub(r'^●\s*', '. ', text, flags=re.MULTILINE)

    # To keep numeric references and legal terminologies intact - customize as necessary
    pattern_to_keep = re.compile(r'\b17Ad-22\(e\)\(7\)\b|\b17Ad-22(e)(19)\b', re.IGNORECASE)
    kept_terms = pattern_to_keep.findall(text)

    # To convert text to lowercase, except for kept_terms
    text = text.lower()

    # Tokenize the text into sentences
    sentences = sent_tokenize(text)
    processed_sentences = []

    for sentence in sentences:
        # Remove punctuation while preserving intra-word dashes
        sentence = re.sub(r'(?<!\w)-|-(?!\w)', ' ', sentence)  # Replace dashes not within words
        sentence = sentence.translate(str.maketrans('', '', string.punctuation))

        # Tokenization
        tokens = word_tokenize(sentence)

        # Remove stopwords
        stop_words = set(stopwords.words('english'))
        filtered_tokens = [word for word in tokens if word not in stop_words]

        # Lemmatization
        lemmatizer = WordNetLemmatizer()
        lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]

        processed_sentences.append(' '.join(lemmatized_tokens))

    processed_text = '. '.join(processed_sentences)

    # Reinsert numeric references and legal terminologies with original casing
    for term in kept_terms:
        term_lowercase = term.lower()
        processed_text = re.sub(re.escape(term_lowercase), term, processed_text)

    return processed_text


pdf_path1 = '/content/rule 7 doc2.pdf'
pdf_path2 = '/content/rule 19 doc 2.pdf'

# Extract and preprocess text
text1 = extract_text_from_pdf(pdf_path1)
text2 = extract_text_from_pdf(pdf_path2)

preprocessed_text1 = preprocess_text_modified(text1)
preprocessed_text2 = preprocess_text_modified(text2)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [3]:
preprocessed_text1

'principle 7 liquidity risk fmi effectively measure monitor manage liquidity risk. fmi maintain sufficient liquid resource relevant currency effect sameday appropriate intraday multiday settlement payment obligation high degree confidence wide range potential stress scenario include limited default participant affiliate would generate largest aggregate liquidity obligation fmi extreme plausible market condition. key consideration 1 fmi robust framework manage liquidity risk participant settlement bank nostro agent custodian bank liquidity provider entity. key consideration 2 fmi effective operational analytical tool identify measure monitor settlement funding flow ongoing timely basis including use intraday liquidity. key consideration 3 payment system ss including one employing dns mechanism maintain sufficient liquid resource relevant currency effect sameday settlement appropriate intraday multiday settlement payment obligation high degree confidence wide range potential stress scena

# **BERT**

In [4]:
import torch
from transformers import BertModel, BertTokenizer
from sentence_transformers import util
import numpy as np

# Initialize the tokenizer and model from Hugging Face Transformers
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Tokenize and encode the texts as PyTorch tensors
inputs1 = tokenizer(preprocessed_text1, return_tensors="pt", padding=True, truncation=True)
inputs2 = tokenizer(preprocessed_text2, return_tensors="pt", padding=True, truncation=True)

# Obtain embeddings
with torch.no_grad():
    outputs1 = model(**inputs1)
    outputs2 = model(**inputs2)

    # Get all token embeddings from the last hidden state
    token_embeddings1 = outputs1.last_hidden_state
    token_embeddings2 = outputs2.last_hidden_state

    # Average the token embeddings to get a single document embedding
    doc_embedding1 = token_embeddings1[:, 1:-1, :].mean(dim=1)
    doc_embedding2 = token_embeddings2[:, 1:-1, :].mean(dim=1)

# Calculate the cosine similarity between the document embeddings
similarity = util.pytorch_cos_sim(doc_embedding1, doc_embedding2)

print(f"cosine similarity is {similarity.item()}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

cosine similarity is 0.9217327237129211


# **SBERT**

In [5]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')

# Split texts into lists of sentences.
sentences1 = preprocessed_text1.split('. ')
sentences2 = preprocessed_text2.split('. ')

# Encode sentences to get their embeddings
embedding1 = model.encode(sentences1, convert_to_tensor=True)
embedding2 = model.encode(sentences2, convert_to_tensor=True)

# Calculate pairwise cosine similarities between all sentences in both sets
similarity_matrix = util.pytorch_cos_sim(embedding1, embedding2)

# Define a similarity threshold for "high similarity"
similarity_threshold = 0.6

# Extract the sentences and scores for pairs above the threshold
high_similarity_pairs = [{
    'Sentences from rule 7': sentences1[i],
    'Sentences from rule 19': sentences2[j],
    'Cosine Similarity Score': similarity_matrix[i, j].item()
} for i in range(similarity_matrix.size(0)) for j in range(similarity_matrix.size(1)) if similarity_matrix[i, j] > similarity_threshold]

high_similarity_df = pd.DataFrame(high_similarity_pairs)

high_similarity_df.sort_values(by='Cosine Similarity Score', ascending=False, inplace=True)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
high_similarity_df.to_csv('Similar_sentences_file2.csv', index=False)


In [7]:
high_similarity_df

,Sentences from rule 7,Sentences from rule 19,Cosine Similarity Score
5,ccas 17ad22e7 covered clearing agency shall es...,ccas 17ad22e19 covered clearing agency shall e...,0.818542
14,respect member sld requirement nscc provides r...,particular nscc requires member submit informa...,0.691035
13,respect member sld requirement nscc provides r...,thereafter part ongoing member due diligence p...,0.686795
3,key consideration 10 fmi establish explicit ru...,key consideration 1 fmi ensure rule procedure ...,0.675546
19,entity act otherwise member thus subject ongoi...,thereafter part ongoing member due diligence p...,0.647622
0,key consideration 7 fmi obtain high degree con...,key consideration 1 fmi ensure rule procedure ...,0.634824
18,entity act otherwise member thus subject ongoi...,ccas 17ad22e19 covered clearing agency shall e...,0.631049
1,key consideration 7 fmi obtain high degree con...,key consideration 3 fmi identify indirect part...,0.628933
6,ccas 17ad22e7 covered clearing agency shall es...,thereafter part ongoing member due diligence p...,0.627752
7,liquidity risk management framework nscc affil...,ccas 17ad22e19 covered clearing agency shall e...,0.620430


# Calculating the overall cosine similarity between both rules using SBERT

In [8]:
# Calculate the average embedding for each document
avg_embedding1 = torch.mean(embedding1, dim=0)
avg_embedding2 = torch.mean(embedding2, dim=0)

# Calculate the cosine similarity between the average embeddings of the two documents
overall_similarity = util.pytorch_cos_sim(avg_embedding1, avg_embedding2)

print("Overall similarity between the two documents:", overall_similarity.item())

Overall similarity between the two documents: 0.7677137851715088


# **TF-IDF**

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# Combine documents into a list for vectorization
documents = [preprocessed_text1, preprocessed_text2]

# Initialize a TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Generate the TF-IDF vectors for the documents
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

# Calculate the cosine similarity between the documents
cos_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])

# Print the cosine similarity score
print(f"Cosine similarity between the documents is: {cos_sim[0][0]}")

Cosine similarity between the documents is: 0.360892005728255
